In [7]:
import pandas as pd
import zipfile
import csv
import random
import time
import numpy as np
from scipy import sparse as sp
from scipy.sparse.linalg import norm
import sklearn.preprocessing as pp
import pickle  

In [2]:
zf = zipfile.ZipFile('h-and-m-personalized-fashion-recommendations.zip') 

In [3]:
#Customers datafile
customers = pd.read_csv(zf.open('customers.csv'))

In [4]:
customers_sub = customers.sample(n=200000, random_state=1, ignore_index= True)

In [5]:
customers_sub.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,525f5b5fae1ed058e47e8a8aedec7188288f9577f6a526...,1.0,1.0,ACTIVE,Regularly,34.0,f99dc9776fa383a4ce9c959b232b282fd0e28bd14bab01...
1,129114245df6f25b7b81ea9cdebc3e6e0ae14d66895935...,NaN,NaN,ACTIVE,NONE,24.0,2c29ae653a9282cce4151bd87643c907644e09541abc28...
2,0cbe03d81ad1038d2223e67d98b964e4a95e14971960da...,NaN,NaN,ACTIVE,NONE,34.0,59c4748b09214a8946e22660d9855179358cf3e55e1b2d...
3,bb455bc38400bd6902f9bea828df555a3d55818fcb144b...,1.0,1.0,ACTIVE,Regularly,54.0,1d98a184bc258bf41ea61b66cdcc46bc5c05c2d982c38e...
4,40460a10de4ab8ac5c3b450ef01d67e600b3832106f552...,NaN,NaN,PRE-CREATE,NONE,50.0,132da0a49be9179999f81aae0daf43757e6e8a70d39b89...


In [6]:
customers_sub_train = customers_sub.sample(n= 140000, random_state = 2, ignore_index = True)

In [7]:
customers_sub_test = customers_sub.loc[~customers_sub.customer_id.isin(customers_sub_train.customer_id)]

In [8]:
print(customers_sub_train.shape)
print(customers_sub_test.shape)

(140000, 7)
(60000, 7)


In [ ]:
transactions_train = pd.read_csv(zf.open('transactions_train.csv'))

In [5]:
train = transactions_train.loc[transactions_train.customer_id.isin(customers_sub_train.customer_id)]
test = transactions_train.loc[transactions_train.customer_id.isin(customers_sub_test.customer_id)]

NameError: name 'customers_sub_train' is not defined

In [13]:
print("train")
print("customers:", train.customer_id.nunique(), "shape:", train.shape, "articles:", train.article_id.nunique())
print("test")
print("customers:", test.customer_id.nunique(), "shape:", test.shape, "articles:", test.article_id.nunique())

train
customers: 139007 shape: (3219393, 5) articles: 86195
test
customers: 59595 shape: (1395892, 5) articles: 74953


In [14]:
# Subset down to items in both sets
# Maybe I shouldnt do this? It deletes only specific transactions from a customer
#train = train.loc[train.article_id.isin(test.article_id.unique())]
#test = test.loc[test.article_id.isin(train.article_id.unique())]

In [15]:
print("train")
print("customers:", train.customer_id.nunique(), "shape:", train.shape, "articles:", train.article_id.nunique())
print("test")
print("customers:", test.customer_id.nunique(), "shape:", test.shape, "articles:", test.article_id.nunique())

train
customers: 139007 shape: (3219393, 5) articles: 86195
test
customers: 59595 shape: (1395892, 5) articles: 74953


In [16]:
train = train.reset_index(drop = True)

In [17]:
train.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,0008968c0d451dbc5a9968da03196fe20051965edde741...,531310002,0.022525,2
1,2018-09-20,0008968c0d451dbc5a9968da03196fe20051965edde741...,529841001,0.020322,2
2,2018-09-20,00401a367c5ac085cb9d4b77c56f3edcabf25153615db9...,613456009,0.016932,2
3,2018-09-20,00401a367c5ac085cb9d4b77c56f3edcabf25153615db9...,633675001,0.010153,2
4,2018-09-20,00401a367c5ac085cb9d4b77c56f3edcabf25153615db9...,648719001,0.025407,2


In [18]:
# Lets put this in a function
def create_subsample(n_customers, train_pct):
    
    customers_sub = customers.sample(n=n_customers, random_state=1, ignore_index= True)
    customers_sub_train = customers_sub.sample(n=int(n_customers*train_pct), random_state = 2, ignore_index = True)
    customers_sub_test = customers_sub.loc[~customers_sub.customer_id.isin(customers_sub_train.customer_id)]
    train = transactions_train.loc[transactions_train.customer_id.isin(customers_sub_train.customer_id)]
    test = transactions_train.loc[transactions_train.customer_id.isin(customers_sub_test.customer_id)]
    train = train.reset_index(drop = True)
    
    return train, test

In [13]:
transactions_train.t_dat = pd.to_datetime(transactions_train.t_dat)

In [14]:
transactions_train.shape

(31788324, 5)

In [15]:
transactions_train = transactions_train.loc[transactions_train.t_dat >= '2019-09-01',:]

In [7]:
## Create train test based on time

def create_train_test(sample_size):
    # sub to frequent shoppers
    customer_trans = transactions_train.customer_id.value_counts()
    customer_trans_large = customer_trans.index[customer_trans >= 20]
    trans_sub = transactions_train.loc[transactions_train.customer_id.isin(customer_trans_large), :]
    
    #sub to popular items
    item_sample = trans_sub.article_id.value_counts()
    item_sample_large = item_sample.index[item_sample >= 1000]
    trans_sub = trans_sub.loc[trans_sub.article_id.isin(item_sample_large), :]
    
    # Split trans to most recent for test
    test = trans_sub.loc[trans_sub.t_dat >= '2020-09-16',:]
    # Sample from remaining customers
    customer_list = list(test.customer_id.unique())
    customer_list = random.sample(customer_list, sample_size)

    #Create train and test
    test = test.loc[test.customer_id.isin(list(customer_list)),:]
    train = trans_sub.loc[trans_sub.customer_id.isin(customer_list),:]
    train = train.loc[trans_sub.t_dat < '2020-09-16',:]
    
    return train, test

In [8]:
train, test = create_train_test(1000)

NameError: name 'transactions_train' is not defined

In [24]:
print(train.shape)
print(test.shape)

(17332, 5)
(1765, 5)


In [25]:
train.article_id.value_counts()

706016001    95
706016003    61
759871002    57
156231001    53
706016002    50
             ..
678342028     1
870611001     1
873564004     1
816521005     1
909370001     1
Name: article_id, Length: 1978, dtype: int64

In [26]:
print(train.shape)
print(train.customer_id.nunique())
print(train.article_id.nunique())

(17332, 5)
995
1978


In [27]:
# Save UU recommendations
import pickle  
filehandler = open("train_small.pkl", 'wb') 
pickle.dump(train, filehandler)
filehandler.close()

filehandler = open("test_small.pkl", 'wb') 
pickle.dump(test, filehandler)
filehandler.close()

In [5]:
filehandler = open("train_small.pkl", "rb")
train = pickle.load(filehandler)
filehandler.close()

In [2]:
### Start Here

filehandler = open("transactions_6mth.pkl", "rb")
transactions_6mth = pickle.load(filehandler)
filehandler.close()
transactions_6mth

,t_dat,customer_id,article_id,price,sales_channel_id
16118327,2019-09-01,02dd164d87472e7311cd12ea8df993b4a084ad528ed4af...,552716001,0.033881,2
16118467,2019-09-01,046c9b461a6f7a92512c73080f9dd85980e2e421a49f11...,562245046,0.033881,2
16118468,2019-09-01,046c9b461a6f7a92512c73080f9dd85980e2e421a49f11...,562245046,0.033881,2
16118668,2019-09-01,06207e084340d4a79aab1bfa0e5b5a63d9b7e89e665112...,160442043,0.013542,1
16119188,2019-09-01,0af4ebb4b23fc037e593c0e9a33cf53fc0ee37dd6ce35c...,711053007,0.013542,1
...,...,...,...,...,...
31788300,2020-09-22,ffc2e7c210e3ea602e6d229116773cc0588c929f8cc70b...,858856002,0.042356,1
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1


In [3]:
test = transactions_6mth.loc[transactions_6mth.t_dat >= '2020-09-16',:]
train = transactions_6mth.loc[transactions_6mth.t_dat < '2020-09-16',:]

In [40]:
print(train.article_id.nunique())
print(test.article_id.nunique())
print(transactions_6mth.article_id.nunique())

6185
6206
6206


In [5]:
customerIds = train.customer_id.unique()
customerIds.sort()
articleIds = train.article_id.unique()
articleIds.sort()

m = customerIds.size
n = articleIds.size
numTrans = len(train)

customerIds_to_customerIdsIDX = dict(zip(customerIds, range(0, customerIds.size )))
customerIDX_to_customerId = dict(zip(range(0, customerIds.size), customerIds))

itemId_to_itemIDX = dict(zip(articleIds, range(0, articleIds.size)))
itemIDX_to_itemId = dict(zip(range(0, articleIds.size), articleIds))

train['rating'] = list(np.full(len(train), 1))

df = pd.concat([train['customer_id'].map(customerIds_to_customerIdsIDX), train['article_id'].map(itemId_to_itemIDX), train['rating']], axis=1)
df.columns = ['customer', 'item', 'rating']
df.sort_values(by = 'customer', inplace = True)


display(df.head())

C:\Users\Blake\AppData\Local\Temp/ipykernel_9540/3202145238.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['rating'] = list(np.full(len(train), 1))


,customer,item,rating
29794828,0,3817,1
31140487,0,5156,1
30180376,0,4502,1
31412223,0,4103,1
31140486,0,5055,1


In [6]:
#df = df.loc[df.customer < 100,:]

In [7]:
df_trans_pic = pd.concat([train['t_dat'], train['customer_id'].map(customerIds_to_customerIdsIDX), train['article_id'].map(itemId_to_itemIDX)], axis=1)
df_trans_pic.columns = ['date','customer', 'item']
df_trans_pic

,customer,item,rating
29794828,0,3817,1
31140487,0,5156,1
30180376,0,4502,1
31412223,0,4103,1
31140486,0,5055,1
...,...,...,...
28223529,11742,2550,1
28516682,11742,1927,1
28516683,11742,4425,1
30916081,11742,2680,1


In [8]:
R = sp.csr_matrix((df.rating, (df.customer, df.item)))
R_dok = R.todok()

In [9]:
m = R.shape[0]
n = R.shape[1]
numRatings = R.count_nonzero()

print("There are", m, "users,", n, "items and", numRatings, "ratings.")

There are 11743 users, 6185 items and 194599 ratings.


In [10]:
R_copy = R.copy()
user_cnts = (R != 0).sum(axis=1).A1
user_norm = np.sqrt(R_copy.power(2).sum(1)).A1

# set scaling of rows with 0 zero to 0
zero_index = user_norm == 0
user_norm[zero_index] = 1
user_scales = 1 / user_norm
user_scales[zero_index] = 0

R_copy.data = R_copy.data * np.repeat(user_scales, user_cnts)

In [11]:
def compute_pairwise_user_similarity(u_id, v_id):
    u = R[u_id,:].copy()
    v = R[v_id,:].copy()
    
    numerator = u.dot(v.T)
    denominator = norm(u)*norm(v)
    #print(u)
    #print(norm(v))
    #print(v)
    
    if denominator == 0:
        similarity = 0.;
    else:
        similarity = (numerator/denominator).A.item()
    
    return similarity

In [12]:
def compute_user_similarities(u_id):
    uU = np.empty((m,))
        
    uU = (R_copy.dot(R_copy[u_id,:].T)).A.flatten()
    
    ##############################
    ## Mat.dot(Vec.T) is actually same as Mat @ Vec.T
    ##############################
    
    
    # YOUR CODE HERE
    #for i in range(uU.size):
    #    uU[i] = compute_pairwise_user_similarity(u_id, i)
    
    #raise NotImplementedError()
    
    return uU

In [13]:
def create_sim_list(u_id):
    uU = compute_user_similarities(u_id)
    uU_copy = uU.copy() ## so that we can modify it, but also keep the original
    
    # YOUR CODE HERE
    uU_copy = np.delete(uU_copy, u_id)

    # Sort array    
    sort_order = np.argsort(uU_copy)[::-1]
    uU_copy = uU_copy[sort_order]

    users_sorted = list(df.customer.unique())
    users_sorted = np.delete(users_sorted, u_id)
    users_sorted = users_sorted[sort_order]
    
    return users_sorted

In [14]:

def get_prediction(uU, users_sorted, i_id, n_neighbor):
    nh_user = []
    nh_sim = []
    
    i = 0
    while len(nh_sim) < n_neighbor:
        user_i = users_sorted[i]
        #print(nh_sim)
        #print(user_i)
    	#uU[user_i])
        if (user_i, i_id) in R_dok:
            nh_sim.append(uU[i])
            nh_user.append(user_i)
        i += 1
        #print(i)
        if i == len(uU):
            break

    prediction = sum(nh_sim)
    
    return prediction

In [15]:
def get_recommendations(u_id, n_neighbor, n_item_recommend):
    # get user similarities
    uU = compute_user_similarities(u_id)
    uU_copy = uU.copy() ## so that we can modify it, but also keep the original

    # Sort arrays by highest sim
    sort_order = np.argsort(uU_copy)[::-1]
    uU_copy = uU_copy[sort_order]
    
    # delete the user from the sim list
    uU_copy = np.delete(uU_copy, 0)

    users_sorted = np.array(df.customer.unique())
    users_sorted = users_sorted[sort_order]
    users_sorted = np.delete(users_sorted, 0)
    
    item_arr = np.array(df.item.unique())
    pred_arr = np.zeros(len(item_arr))
    k = n_neighbor
    i = 0
    for i_id in item_arr:
        #print(u_id, i_id)
        prediction = get_prediction(uU_copy, users_sorted, i_id, n_neighbor)
        pred_arr[i] = prediction
        i += 1
    
    item_sort_order = np.argsort(pred_arr)[::-1]
    item_arr = item_arr[item_sort_order][0:n_item_recommend]
    
    return item_arr

In [17]:
### FOR OTHER REC PROJECT!!!!!!!
'''
def get_recommendations(u_id, n_neighbor, n_item_recommend):
    # get user similarities
    uU = compute_user_similarities(u_id)
    uU_copy = uU.copy() ## so that we can modify it, but also keep the original

    # Sort arrays by highest sim
    sort_order = np.argsort(uU_copy)[::-1]
    uU_copy = uU_copy[sort_order]
    
    # delete the user from the sim list
    uU_copy = np.delete(uU_copy, 0)

    users_sorted = np.array(df.customer.unique())
    users_sorted = users_sorted[sort_order]
    users_sorted = np.delete(users_sorted, 0)
    user_list = users_sorted[:100]
    
    return user_list
'''    

In [34]:
start = time.perf_counter()

rec_dict = {}

customer_list = df.customer.unique()[500:700]

for u_id in customer_list:
    recs = get_recommendations(u_id, n_neighbor = 5, n_item_recommend = 12)
    rec_dict[u_id] = recs

end = time.perf_counter()
end - start    
    

3191.5678844

In [23]:
#recs = get_recommendations(1, n_neighbor = 5, n_item_recommend = 12)

In [24]:
rec_dict

{0: array([1174, 2412, 4290, 4339, 3694, 3841, 5199, 4296, 5955, 3202, 5971,
        4916], dtype=int64),
 1: array([ 881, 1290, 1366, 1797, 4879, 1371, 1300,  413, 1716, 4189,  766,
         839], dtype=int64),
 2: array([2045, 2047,  308,  148, 3545, 2053, 1047,  967, 1146, 3242, 1291,
        2046], dtype=int64),
 3: array([ 398, 4706, 2804, 1047, 1211,  144, 2445, 5667, 2961, 5785, 2045,
        1003], dtype=int64),
 4: array([ 881, 1290, 1448, 2997, 4189,  413,  766,  839, 1716, 1300, 2182,
        2186], dtype=int64),
 5: array([ 881, 1290, 4919, 2997, 2548, 3482, 2107, 4106, 4641, 1938,  331,
        4097], dtype=int64),
 6: array([ 176, 2515, 2735, 3385, 5462, 3892, 1163, 3612, 4996, 4999,  767,
        4814], dtype=int64),
 7: array([1508, 5604, 4351, 1098, 2250, 4974, 3483, 5665,  454, 5109, 3499,
        2293], dtype=int64),
 8: array([1047, 2045, 1705, 5345, 1532,  148, 4467,  967, 5578, 1922, 4443,
        5604], dtype=int64),
 9: array([1461,  785, 3007, 1047, 1705, 5345,

In [ ]:
# Save UU recommendations
import pickle  
filehandler = open("uu_dict.pkl", 'wb') 
pickle.dump(rec_dict, filehandler)
filehandler.close()

In [110]:
# Open UU recommendations
filehandler = open("uu_dict.pkl", "rb")
output = pickle.load(filehandler)
filehandler.close()


dict

In [29]:
def calc_cust_precision(u_id):
    ground_truth = cust_orders[u_id]
    hits_list = (np.isin(rec_dict[u_id], ground_truth))*1
    ap = 0 ## average precision
    hits = np.sum(hits_list)
    relevant_n = len(ground_truth)

    if (hits != 0):
        for rank in range(1,len(hits_list)+1):
            # update ap on hit items
            hit = hits_list[rank-1]
            if hit:
                rank_hit_list = hits_list[0:rank]
                prec_i = np.mean(rank_hit_list)
                ap += prec_i  
        ap /= (min(relevant_n, 12))
    
    return ap

In [30]:
test_df = pd.concat([test['customer_id'].map(customerIds_to_customerIdsIDX), test['article_id'].map(itemId_to_itemIDX)], axis=1)
test_df.columns = ['customer', 'item']
test_df.sort_values(by = 'customer', inplace = True)

cust_orders = test_df.groupby("customer")["item"].unique()

display(test_df.loc[test_df.customer == 1])

,customer,item
31723329,1.0,1795.0
31723330,1.0,593.0


In [35]:
ap_u = 0.

for u_id in customer_list:
    ap_u += calc_cust_precision(u_id)
mean_ap = ap_u / len(customer_list)       

In [38]:
mean_ap

0.05023826058201057

In [37]:
sum_ap_500_700 = mean_ap*len(customer_list)
#total

filehandler = open("sum_ap_500_700", 'wb') 
pickle.dump(sum_ap_500_700, filehandler)
filehandler.close()

In [44]:
#filehandler = open(str(x), "rb")
#sum_ap_0_test = pickle.load(filehandler)
#filehandler.close()
#print(sum_ap_0_test)

1


In [29]:
# get all of the saved files
sum_list = []

file_list = ['sum_ap_100', 'sum_ap_200', 'sum_ap_500_700', 'sum_ap_700_1000', 'sum_ap_1000_1500', 'sum_ap_9000_11743']
for file in file_list:
    
    filehandler = open(file + '.pkl', "rb")
    sum_i = pickle.load(filehandler)
    filehandler.close()
    sum_list.append(sum_i)


In [31]:
# Check if each metric seems reasonable
div_list = [100,300,200,300,500,11743-9000]

res = [i / j for i, j in zip(sum_list, div_list)]
print(sum_list)
print(div_list)
print(res)

[6.056878306878306, 15.61100609267276, 10.047652116402114, 17.820128066378068, 20.8424062049062, 139.6703869047619]
[100, 300, 200, 300, 500, 2743]
[0.060568783068783064, 0.05203668697557587, 0.05023826058201057, 0.05940042688792689, 0.0416848124098124, 0.050918843202610975]


In [32]:
np.sum(sum_list)/np.sum(div_list)

0.05069960359449659

In [39]:
type(purchases)

pandas.core.series.Series

In [70]:
hit_list

array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1])

In [71]:
ap

0.2611111111111111

In [10]:
import pickle
filehandler = open("train_small.pkl", "rb")
train = pickle.load(filehandler)
filehandler.close()

In [11]:
train

,customer,item,rating
31140485,0,6233,1
29794828,0,4539,1
29794827,0,2173,1
29794829,0,4539,1
30180375,0,6468,1
...,...,...,...
30669401,999,6184,1
26156004,999,817,1
31105332,999,3463,1
26156002,999,3262,1


In [42]:
import sys
  
  
print("User Current Version:-", sys.version)

User Current Version:- 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]


In [4]:
item_purchases = transactions_6mth['article_id'].value_counts()

In [6]:
print(len(item_purchases))
print(sum(item_purchases))

6206
278334


In [13]:
print(sum(item_purchases[:3000])/sum(item_purchases))

0.8447297132222438


In [14]:
12000*1000

12000000